<a href="https://colab.research.google.com/github/HzcIrving/Deep-Learning-for-myself/blob/master/Dynamic_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ---动态规划的三个问题---
1. Policy Evaluation 策略估计问题;
  - Bellman Expectation Equation 
2. Policy Iteration 策略迭代问题;
  - Bellman ... & Greedy Policy Improvement
  - 包括：
    - Policy Evaluation
    - Policy Improvement 
3. Value Iteration 值迭代问题;
  - Bellman Optimally Equation 
4. 主要场景，David Silver大神在动态规划章节中的SMALL GRIDWORLD场景，可以去课件看下...
5. 注意，动态规划与强化学习不同，主要解决的是Planning问题，即系统的环境MDP信息是完全可知的。

In [1]:
# 装在google drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 工作区
import os 
os.chdir('/content/drive/My Drive/DEMO')
WORKSPACE = os.getcwd()

In [4]:
# Git 仓库
!git clone https://github.com/dennybritz/reinforcement-learning.git

Cloning into 'reinforcement-learning'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1274 (delta 0), reused 0 (delta 0), pack-reused 1270
Receiving objects: 100% (1274/1274), 5.25 MiB | 3.52 MiB/s, done.
Resolving deltas: 100% (819/819), done.


### 1.Policy Evaluation问题
在Policy_evaluation中：
  - 在给定环境和环境动态的完整描述的情况下评估策略；
  - policy:[S,A]matrix代表policy
  - env:OpenAI环境；
    - env.P代表代表环境的转移概率。
    - env.P[s][a]是转移矩阵的列表(prob,next_state,reward,done)
    - env.nS代表环境中state数量;
    - env.nA代表环境中action数量(东、南、西、北)
  - theta: value function改变小于theta，停止评估;
  - discount_factor: Gamma 
  - return: Value Function 

In [8]:
# 导入相关库
from IPython.core.debugger import set_trace
import numpy as np
import pprint
from reinforcementlearning.lib.envs.gridworld import GridworldEnv

import warnings 
warnings.filterwarnings('ignore') #忽略警告

pp = pprint.PrettyPrinter(indent=2) # 方便打印
env = GridworldEnv() #Gridworld环境

def policy_evaluation(policy,env,discount_factor=1.0,theta=0.00001):
  # 初始化,all 0 
  # 注意，策略: n,e,s,w都是.25
  V = np.zeros(env.nS)
  while True:
    delta = 0 
    # 对于每一个state,执行"full backup"
    for s in range(env.nS): 
      v = 0
      # 查看可能的后续动作（已知MDP的基础上才可以）
      # action_prob -> policy π
      for a,action_prob in enumerate(policy[s]):
        # 对于每一个动作，查看可能产生的所有next_state
        for prob,next_state,reward,done in env.P[s][a]:
          # 计算期望Value
          v += action_prob*prob*(reward+discount_factor*V[next_state])
      
      # 计算值函数改变了多少
      delta = max(delta,np.abs(v-V[s]))
      V[s] = v
    # stop条件
    if delta < theta:
      break 
  return np.array(V)

random_policy = np.ones([env.nS, env.nA]) / env.nA #均匀random policy
v = policy_evaluation(random_policy, env)

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Reshaped Grid Value Function:
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]



### 2.Policy Iteration问题
- 先估计policy π的好坏(Policy Eval);
- 在根据贪婪策略选择选择$v_π$最大的π;
- Policy_Evaluation:过程同上面Policy_Evaluation;
- Policy_improvement:反复评估和改进策略直到找到最佳策略为止;
  - env:环境
  - policy_eval_fn:PolicyEvaluationFunction，这是一个迭代过程;
  - discount_factor
  - return:
    - 一个元组(policy,V)
    - policy是最优policy,策略是最优策略，形状为[S，A]的矩阵，其中每个状态s包含动作的有效概率分布;
    - V是最优策略的价值函数

In [20]:
import numpy as np
import pprint
import sys
# if "../" not in sys.path:
#   sys.path.append("../") 
from reinforcementlearning.lib.envs.gridworld import GridworldEnv

pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

print("环境维数4x4grid:",env.shape)
print("动作维数:",env.nA)
print("状态维数:",env.nS)

def policy_evaluation(policy,env,discount_factor=1.0,theta=0.00001):
  # 初始化,all 0 
  # 注意，策略: n,e,s,w都是.25
  V = np.zeros(env.nS)
  while True:
    delta = 0 
    # 对于每一个state,执行"full backup"
    for s in range(env.nS): 
      v = 0
      # 查看可能的后续动作（已知MDP的基础上才可以）
      # action_prob -> policy π
      for a,action_prob in enumerate(policy[s]):
        # 对于每一个动作，查看可能产生的所有next_state
        for prob,next_state,reward,done in env.P[s][a]:
          # 计算期望Value
          v += action_prob*prob*(reward+discount_factor*V[next_state])
      
      # 计算值函数改变了多少
      delta = max(delta,np.abs(v-V[s]))
      V[s] = v
    # stop条件
    if delta < theta:
      break 
  return np.array(V)

def policy_improvement(env,policy_eval_fn=policy_evaluation,discount_factor=1.0):
  def one_step_lookahead(state,V):
    """
    辅助函数可计算给定状态下所有动作的值。
    Args:
      state:要考虑的状态（int）
      V:用作估计量的值，长度为env.nS的向量
    Return:
      一个长度为env.nA的向量，其中包含每个动作的期望值。
    """
    A = np.zeros(env.nA)
    for a in range(env.nA):
      for prob,next_state,reward,done in env.P[state][a]:
        A[a] += prob*(reward+discount_factor*V[next_state])
    return A 
  
  # 从一个随机策略开始
  policy = np.ones([env.nS,env.nA])/env.nA # 0.25
  while True:
    # 评估当前策略
    V = policy_eval_fn(policy,env,discount_factor)
    policy_stable = True # 若要更改策略，设置为False
    
    # 对于每一个状态
    for s in range(env.nS):
      # 我们将在当前政策下采取最佳措施
      chosen_a = np.argmax(policy[s])
      # 通过one-step lookahead来找到最佳动作
      action_values = one_step_lookahead(s,V)
      best_a = np.argmax(action_values)

      # 贪婪更新policy
      if chosen_a != best_a:
        policy_stable = False 
      
      # 生成维数为nA的对角矩阵,选择第best_a行
      policy[s] = np.eye(env.nA)[best_a]
    
    # 如果是stable的
    if policy_stable:
      return policy,V


# policy, v = policy_improvement(env)
# print("Policy Probability Distribution:")
# print(policy)
# print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

# print("Value Function:")
# print(v)
# print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

环境维数: [4, 4]
动作维数: 4
状态维数: 16
Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



### 3.Value Iteration问题

In [22]:
import numpy as np
import pprint
import sys
# if "../" not in sys.path:
#   sys.path.append("../") 
from reinforcementlearning.lib.envs.gridworld import GridworldEnv

pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

print("环境维数4x4grid:",env.shape)
print("动作维数:",env.nA)
print("状态维数:",env.nS)

def value_iteration(env,theta=0.0001,discount_factor=1.0):
  def one_step_lookahead(state,V):
    """
    辅助函数可计算给定状态下所有动作的值。
    Args:
      state:要考虑的状态（int）
      V:用作估计量的值，长度为env.nS的向量
    Return:
      一个长度为env.nA的向量，其中包含每个动作的期望值。
    """
    A = np.zeros(env.nA)
    for a in range(env.nA):
      for prob,next_state,reward,done in env.P[state][a]:
        A[a] += prob*(reward+discount_factor*V[next_state])
    return A 

  V = np.zeros(env.nS) # 初始化V值表
  while True:
    # 停止条件
    delta = 0 
    # 更新each state...
    for s in range(env.nS):
      # 一次one-step lookahead来找到最佳action
      A = one_step_lookahead(s,V)
      best_action_value = np.max(A) #找最大Value
      # 计算delta
      delta = max(delta, np.abs(best_action_value - V[s]))
      V[s] = best_action_value
    
    if delta<theta:
      break 
  
  # 建立一个确定性策略(使用最优值函数)
  policy = np.zeros([env.nS,env.nA]) # 16x4 
  for s in range(env.nS):
    # 找到best action 
    A = one_step_lookahead(s,V)
    best_action = np.argmax(A)
    # 总采用最优策略
    policy[s,best_action] = 1.0
  
  return policy,V 

policy, v = value_iteration(env)
print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")
print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

环境维数4x4grid: [4, 4]
动作维数: 4
状态维数: 16
Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

